<a href="https://colab.research.google.com/github/emkrit1242/MSC_DPDM_2025/blob/main/Frequent_Patterns_(Association_Rules).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/dpdm/data'

In [5]:
data = pd.read_csv(os.path.join(path,'reduced_marketbasket.csv'),encoding='latin-1') #encoding= 'latin-1' bc. มีภาษาที่เป็นจุด ๆ บนหัวทำให้ข้อมมูลเละ

In [6]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/09/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/09/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/09/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/09/2011 12:50,4.15,12680.0,France


In [ ]:
data.isnull().any()

,0
InvoiceNo,False
StockCode,False
Description,True
Quantity,False
InvoiceDate,False
UnitPrice,False
CustomerID,True
Country,False


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## การเตรียมข้อมูล for Apriori Algorithm

ข้อมูลที่ใช้มักจะอยู่ในรูปแบบของธุรกรรม (Transactions) โดยแต่ละธุรกรรมจะประกอบไปด้วยชุดของรายการ (Items)

InvoiceNo = รหัสใบเสร็จ, StockCode = รหัสสินค้า, Description = คำอธิบายรายการสินค้า, Quantity = จำนวนชิ้น, InvoiceDate = วันที่ซื้อ, UnitPrice = ราคาต่อหน่วย, CustomerID = รหัสสมาชิกลูกค้า, Country = ประเทศที่ตั้ง กรณีมีหลายสาขา

In [7]:
data.groupby('InvoiceNo')['StockCode'].apply(list) #รวม ให้สินค้ารหัสใบเสร็จเดียวกันมาอยู่ด้วยกัน # จัดกลุ่มข้อมูล .apply(func  ที่จะทำ)

,StockCode
InvoiceNo,
536365,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,..."
536366,"[22633, 22632]"
536367,"[84879, 22745, 22748, 22749, 22310, 84969, 226..."
536368,"[22960, 22913, 22912, 22914]"
536369,[21756]
...,...
C581484,[23843]
C581490,"[22178, 23144]"
C581499,[M]


In [8]:
transactions = data.groupby('InvoiceNo')['StockCode'].apply(list)

In [9]:
len(data['Country'].unique())

38

In [10]:
set(data['Country']) #  เหมือนกับ .unique() สามารถระบุได้ไหมว่าลูกค้าประเทศไหนซื้อสินค้าอะไร

{'Australia',
 'Austria',
 'Bahrain',
 'Belgium',
 'Brazil',
 'Canada',
 'Channel Islands',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'EIRE',
 'European Community',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hong Kong',
 'Iceland',
 'Israel',
 'Italy',
 'Japan',
 'Lebanon',
 'Lithuania',
 'Malta',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'RSA',
 'Saudi Arabia',
 'Singapore',
 'Spain',
 'Sweden',
 'Switzerland',
 'USA',
 'United Arab Emirates',
 'United Kingdom',
 'Unspecified'}

## Association Rules

In [11]:
!pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=07d6d3689cd629e59a427e708b0bb5884805a2328b2551ae69bc75d5550c4d18
  Stored in directory: /root/.cache/pip/wheels/7f/49/e3/42c73b19a264de37129fadaa0c52f26cf50e87de08fb9804af
Successfully built apyori


In [12]:
from apyori import apriori

In [13]:
list(apriori(transactions,min_support = 0.5))

[]

In [14]:
list(apriori(transactions,min_support = 0.1))

[]

In [15]:
frequentItemsets = list(apriori(transactions,min_support = 0.065))
frequentItemsets

[RelationRecord(items=frozenset({'22423'}), support=0.08386100386100385, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'22423'}), confidence=0.08386100386100385, lift=1.0)]),
 RelationRecord(items=frozenset({'47566'}), support=0.06586872586872587, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'47566'}), confidence=0.06586872586872587, lift=1.0)]),
 RelationRecord(items=frozenset({'85099B'}), support=0.08243243243243244, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'85099B'}), confidence=0.08243243243243244, lift=1.0)]),
 RelationRecord(items=frozenset({'85123A'}), support=0.08671814671814672, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'85123A'}), confidence=0.08671814671814672, lift=1.0)])]

In [16]:
frequentItemsets[0]

RelationRecord(items=frozenset({'22423'}), support=0.08386100386100385, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'22423'}), confidence=0.08386100386100385, lift=1.0)])

In [ ]:
frequentItemsets = list(apriori(transactions,min_support = 0.03))
frequentItemsets

[RelationRecord(items=frozenset({'20712'}), support=0.033667953667953665, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'20712'}), confidence=0.033667953667953665, lift=1.0)]),
 RelationRecord(items=frozenset({'20719'}), support=0.03254826254826255, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'20719'}), confidence=0.03254826254826255, lift=1.0)]),
 RelationRecord(items=frozenset({'20724'}), support=0.04054054054054054, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'20724'}), confidence=0.04054054054054054, lift=1.0)]),
 RelationRecord(items=frozenset({'20725'}), support=0.062084942084942085, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'20725'}), confidence=0.062084942084942085, lift=1.0)]),
 RelationRecord(items=frozenset({'20726'}), support=0.04003861003861004, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozense

In [ ]:
len(frequentItemsets)

80

In [ ]:
frequentItemsets[-1]

RelationRecord(items=frozenset({'22697', '22699'}), support=0.03027027027027027, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'22697', '22699'}), confidence=0.03027027027027027, lift=1.0), OrderedStatistic(items_base=frozenset({'22697'}), items_add=frozenset({'22699'}), confidence=0.7417218543046358, lift=17.1523178807947), OrderedStatistic(items_base=frozenset({'22699'}), items_add=frozenset({'22697'}), confidence=0.7, lift=17.1523178807947)])

#HW8 หา Pattern ที่น่าสนใจใน data ตำรวจ

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
Street_london=pd.read_csv('/content/drive/MyDrive/dpdm/data/2025-01/2025-01-city-of-london-street.csv')
Street_london

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,8c7661d1b68d476454c5b68a58daee0d91781a94f60015...,2025-01,City of London Police,City of London Police,-0.107682,51.517786,On or near B521,E01000917,Camden 027C,Other theft,Investigation complete; no suspect identified,NaN
1,1b7fbc8deac5182e6b1e580ab4eb4ed520df688c3576bc...,2025-01,City of London Police,City of London Police,-0.111596,51.518281,On or near Chancery Lane,E01000914,Camden 028B,Theft from the person,Investigation complete; no suspect identified,NaN
2,8476f32b188fae2d0d14b7db79e872fd7688f064e8ced3...,2025-01,City of London Police,City of London Police,-0.097078,51.519045,On or near A1,E01000001,City of London 001A,Drugs,Further investigation is not in the public int...,NaN
3,92b8de6e45c3b711e802fb9d99e2a030c3f56b1c589e55...,2025-01,City of London Police,City of London Police,-0.097290,51.521575,On or near Fann Street,E01000001,City of London 001A,Other theft,Investigation complete; no suspect identified,NaN
4,023587ed2c674a28bd52d6e03d505c3b0dba6d25e8b95b...,2025-01,City of London Police,City of London Police,-0.098519,51.517332,On or near Little Britain,E01000001,City of London 001A,Other theft,Investigation complete; no suspect identified,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
624,1b38bc565a82962512e35c4fe4608f1c18d6c03c942a98...,2025-01,City of London Police,City of London Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN
625,57cb7b896da3bfafbf18dba76e09b10fa80b4abd806b3d...,2025-01,City of London Police,City of London Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN
626,5ff8ee94b9065b413961b58f695f044c2f1774f4454cdb...,2025-01,City of London Police,City of London Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN
627,ad381068dd8a02eccf8127db404f0f5c5ae9837e13dfba...,2025-01,City of London Police,City of London Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN


In [ ]:
S=Street_london.dropna(subset=['LSOA name'])
S

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,8c7661d1b68d476454c5b68a58daee0d91781a94f60015...,2025-01,City of London Police,City of London Police,-0.107682,51.517786,On or near B521,E01000917,Camden 027C,Other theft,Investigation complete; no suspect identified,NaN
1,1b7fbc8deac5182e6b1e580ab4eb4ed520df688c3576bc...,2025-01,City of London Police,City of London Police,-0.111596,51.518281,On or near Chancery Lane,E01000914,Camden 028B,Theft from the person,Investigation complete; no suspect identified,NaN
2,8476f32b188fae2d0d14b7db79e872fd7688f064e8ced3...,2025-01,City of London Police,City of London Police,-0.097078,51.519045,On or near A1,E01000001,City of London 001A,Drugs,Further investigation is not in the public int...,NaN
3,92b8de6e45c3b711e802fb9d99e2a030c3f56b1c589e55...,2025-01,City of London Police,City of London Police,-0.097290,51.521575,On or near Fann Street,E01000001,City of London 001A,Other theft,Investigation complete; no suspect identified,NaN
4,023587ed2c674a28bd52d6e03d505c3b0dba6d25e8b95b...,2025-01,City of London Police,City of London Police,-0.098519,51.517332,On or near Little Britain,E01000001,City of London 001A,Other theft,Investigation complete; no suspect identified,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
562,68814776c3af9455308af6b76748c9bef8cf10975bce80...,2025-01,City of London Police,City of London Police,-0.077597,51.518907,On or near Steward Street,E01004307,Tower Hamlets 015B,Shoplifting,Investigation complete; no suspect identified,NaN
563,d2625d565f9f839b9eb74779d2ca6a7e5a5f0bdc3e286a...,2025-01,City of London Police,City of London Police,-0.057886,51.518735,On or near Raven Row,E01004322,Tower Hamlets 017A,Violence and sexual offences,Under investigation,NaN
564,573205f721777c08d2ec52fd17176d52807ae48769ebf9...,2025-01,City of London Police,City of London Police,-0.062335,51.514762,On or near Hessel Street,E01004321,Tower Hamlets 021C,Shoplifting,Unable to prosecute suspect,NaN
565,a0bb4cb12396b7fb0bb922ef978e7230bed32eaa97982a...,2025-01,City of London Police,City of London Police,-0.068095,51.509785,On or near Dock Street,E01035697,Tower Hamlets 026F,Drugs,Local resolution,NaN


In [ ]:
S['LSOA name'].unique()

array(['Camden 027C', 'Camden 028B', 'City of London 001A',
       'City of London 001B', 'City of London 001C',
       'City of London 001E', 'City of London 001F',
       'City of London 001G', 'Hackney 028A', 'Hackney 033C',
       'Islington 023D', 'Southwark 035A', 'Tower Hamlets 005D',
       'Tower Hamlets 015B', 'Tower Hamlets 017A', 'Tower Hamlets 021C',
       'Tower Hamlets 026F', 'Tower Hamlets 027B'], dtype=object)

In [ ]:
T = S.groupby('LSOA name')['Crime type'].apply(list)
T

,Crime type
LSOA name,
Camden 027C,[Other theft]
Camden 028B,[Theft from the person]
City of London 001A,"[Drugs, Other theft, Other theft, Public order..."
City of London 001B,"[Bicycle theft, Bicycle theft, Drugs, Drugs, D..."
City of London 001C,"[Other theft, Public order, Robbery, Robbery, ..."
City of London 001E,"[Bicycle theft, Criminal damage and arson, Cri..."
City of London 001F,"[Bicycle theft, Bicycle theft, Bicycle theft, ..."
City of London 001G,"[Anti-social behaviour, Bicycle theft, Bicycle..."
Hackney 028A,[Other theft]


In [ ]:
from apyori import apriori

In [ ]:
F=list(apriori(T,min_support = 0.3))
F

[RelationRecord(items=frozenset({'Drugs'}), support=0.3888888888888889, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Drugs'}), confidence=0.3888888888888889, lift=1.0)]),
 RelationRecord(items=frozenset({'Other theft'}), support=0.5, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Other theft'}), confidence=0.5, lift=1.0)]),
 RelationRecord(items=frozenset({'Public order'}), support=0.3888888888888889, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Public order'}), confidence=0.3888888888888889, lift=1.0)]),
 RelationRecord(items=frozenset({'Shoplifting'}), support=0.3888888888888889, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Shoplifting'}), confidence=0.3888888888888889, lift=1.0)]),
 RelationRecord(items=frozenset({'Theft from the person'}), support=0.4444444444444444, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_a

In [ ]:
F[-2]

RelationRecord(items=frozenset({'Public order', 'Theft from the person'}), support=0.3333333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Public order', 'Theft from the person'}), confidence=0.3333333333333333, lift=1.0), OrderedStatistic(items_base=frozenset({'Public order'}), items_add=frozenset({'Theft from the person'}), confidence=0.8571428571428571, lift=1.9285714285714286), OrderedStatistic(items_base=frozenset({'Theft from the person'}), items_add=frozenset({'Public order'}), confidence=0.75, lift=1.9285714285714286)])

มีโอกาส 85.7% ที่เกิดอาชญากรรมประเภท Public order แล้วจะเกิดอาชญากรรมประเภท Theft from the person ด้วย
และมีโอกาส 75% ที่เกิดอาชญากรรมประเภท Theft from the person แล้วจะเกิดอาชญากรรมประเภท Public order ด้วย

#Association Rules (ต่อ)

In [21]:
data_SS=pd.read_csv('/content/drive/MyDrive/dpdm/data/2025-01/2025-01-nottinghamshire-stop-and-search.csv')
data_SS

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing
0,Person search,2025-01-01T00:39:00+00:00,False,NaN,54.007280,-2.896877,Male,10-17,Other ethnic group - Not stated,Black,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,A no further action disposal,False,False
1,Person search,2025-01-01T02:00:00+00:00,False,NaN,53.969736,-2.865897,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest,True,False
2,Person search,2025-01-01T02:08:00+00:00,False,NaN,53.969736,-2.865897,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False
3,Person search,2025-01-01T02:13:00+00:00,False,NaN,53.969736,-2.865897,Male,10-17,Asian/Asian British - Any other Asian background,Other,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False
4,Person search,2025-01-01T02:25:00+00:00,False,NaN,54.007280,-2.896877,Male,over 34,Other ethnic group - Not stated,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,Person search,2025-01-31T16:35:00+00:00,False,NaN,53.978977,-2.605049,Male,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Police and Criminal Evidence Act 1984 (section 1),Stolen goods,Community resolution,True,False
324,Person search,2025-01-31T16:48:00+00:00,False,NaN,54.004503,-2.306429,Female,25-34,Other ethnic group - Not stated,White,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,A no further action disposal,False,False
325,Person and Vehicle search,2025-01-31T20:43:00+00:00,False,NaN,54.007280,-2.896877,Male,18-24,Black/African/Caribbean/Black British - Any ot...,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Community resolution,True,False
326,Person and Vehicle search,2025-01-31T22:31:00+00:00,False,NaN,54.009872,-2.857060,Male,18-24,Other ethnic group - Not stated,Black,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False


In [22]:
data_SS['Transaction'] = data_SS[['Gender', 'Age range', 'Self-defined ethnicity', 'Officer-defined ethnicity', 'Object of search']].values.tolist()
display(data_SS.head())

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,Transaction
0,Person search,2025-01-01T00:39:00+00:00,False,NaN,54.007280,-2.896877,Male,10-17,Other ethnic group - Not stated,Black,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,A no further action disposal,False,False,"[Male, 10-17, Other ethnic group - Not stated,..."
1,Person search,2025-01-01T02:00:00+00:00,False,NaN,53.969736,-2.865897,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest,True,False,"[Male, 10-17, Other ethnic group - Not stated,..."
2,Person search,2025-01-01T02:08:00+00:00,False,NaN,53.969736,-2.865897,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False,"[Male, 10-17, Other ethnic group - Not stated,..."
3,Person search,2025-01-01T02:13:00+00:00,False,NaN,53.969736,-2.865897,Male,10-17,Asian/Asian British - Any other Asian background,Other,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False,"[Male, 10-17, Asian/Asian British - Any other ..."
4,Person search,2025-01-01T02:25:00+00:00,False,NaN,54.007280,-2.896877,Male,over 34,Other ethnic group - Not stated,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False,"[Male, over 34, Other ethnic group - Not state..."


In [29]:
Transaction = data_SS[['Gender', 'Age range', 'Self-defined ethnicity', 'Officer-defined ethnicity', 'Object of search']].values.tolist()


In [28]:
data_SS['L0I'] = Transaction

In [35]:
data_SS_transaction = data_SS[['Date','L0I']]
data_SS_transaction

,Date,L0I
0,2025-01-01T00:39:00+00:00,"[Male, 10-17, Other ethnic group - Not stated,..."
1,2025-01-01T02:00:00+00:00,"[Male, 10-17, Other ethnic group - Not stated,..."
2,2025-01-01T02:08:00+00:00,"[Male, 10-17, Other ethnic group - Not stated,..."
3,2025-01-01T02:13:00+00:00,"[Male, 10-17, Asian/Asian British - Any other ..."
4,2025-01-01T02:25:00+00:00,"[Male, over 34, Other ethnic group - Not state..."
...,...,...
323,2025-01-31T16:35:00+00:00,"[Male, 18-24, White - English/Welsh/Scottish/N..."
324,2025-01-31T16:48:00+00:00,"[Female, 25-34, Other ethnic group - Not state..."
325,2025-01-31T20:43:00+00:00,"[Male, 18-24, Black/African/Caribbean/Black Br..."
326,2025-01-31T22:31:00+00:00,"[Male, 18-24, Other ethnic group - Not stated,..."


In [ ]:
data_SS_transaction.groupby('Date')['L0I'].apply(list)

In [41]:
list_apri=list(apriori(Transaction,min_support = 0.05))

In [57]:
list_apri[:5]

[RelationRecord(items=frozenset({'10-17'}), support=0.20121951219512196, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'10-17'}), confidence=0.20121951219512196, lift=1.0)]),
 RelationRecord(items=frozenset({'18-24'}), support=0.20121951219512196, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24'}), confidence=0.20121951219512196, lift=1.0)]),
 RelationRecord(items=frozenset({'25-34'}), support=0.18597560975609756, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'25-34'}), confidence=0.18597560975609756, lift=1.0)]),
 RelationRecord(items=frozenset({'Article for use in theft'}), support=0.8567073170731707, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Article for use in theft'}), confidence=0.8567073170731707, lift=1.0)]),
 RelationRecord(items=frozenset({'Asian'}), support=0.08231707317073171, ordered_statistics=[OrderedStatistic(items_base

In [43]:
list_apri[0]

RelationRecord(items=frozenset({'10-17'}), support=0.20121951219512196, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'10-17'}), confidence=0.20121951219512196, lift=1.0)])

In [48]:
list_apri[0][0]

frozenset({'10-17'})

In [56]:
list(list_apri[0][0])[0]

'10-17'

In [49]:
dir(list_apri[0][0])

['__and__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__rand__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__rsub__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__xor__',
 'copy',
 'difference',
 'intersection',
 'isdisjoint',
 'issubset',
 'issuperset',
 'symmetric_difference',
 'union']

##หาว่าตำรวจ Black มีความสัมพันธ์กับอะไรบ้าง

In [58]:
police_black=list()
for result in list_apri:
  if 'Black' in result[0]:
    police_black.append(result)

In [62]:
police_black

[RelationRecord(items=frozenset({'Black'}), support=0.17682926829268292, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Black'}), confidence=0.17682926829268292, lift=1.0)]),
 RelationRecord(items=frozenset({'10-17', 'Black'}), support=0.051829268292682924, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'10-17', 'Black'}), confidence=0.051829268292682924, lift=1.0), OrderedStatistic(items_base=frozenset({'10-17'}), items_add=frozenset({'Black'}), confidence=0.25757575757575757, lift=1.4566353187042842), OrderedStatistic(items_base=frozenset({'Black'}), items_add=frozenset({'10-17'}), confidence=0.29310344827586204, lift=1.456635318704284)]),
 RelationRecord(items=frozenset({'18-24', 'Black'}), support=0.057926829268292686, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24', 'Black'}), confidence=0.057926829268292686, lift=1.0), OrderedStatistic(items_base=frozenset({'18-24'}), 

In [63]:
len(police_black)

12

In [64]:
for rule in police_black:
  print(rule)

RelationRecord(items=frozenset({'Black'}), support=0.17682926829268292, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Black'}), confidence=0.17682926829268292, lift=1.0)])
RelationRecord(items=frozenset({'10-17', 'Black'}), support=0.051829268292682924, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'10-17', 'Black'}), confidence=0.051829268292682924, lift=1.0), OrderedStatistic(items_base=frozenset({'10-17'}), items_add=frozenset({'Black'}), confidence=0.25757575757575757, lift=1.4566353187042842), OrderedStatistic(items_base=frozenset({'Black'}), items_add=frozenset({'10-17'}), confidence=0.29310344827586204, lift=1.456635318704284)])
RelationRecord(items=frozenset({'18-24', 'Black'}), support=0.057926829268292686, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24', 'Black'}), confidence=0.057926829268292686, lift=1.0), OrderedStatistic(items_base=frozenset({'18-24'}), items

In [65]:
for rule in enumerate(police_black):
  print(rule)

(0, RelationRecord(items=frozenset({'Black'}), support=0.17682926829268292, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Black'}), confidence=0.17682926829268292, lift=1.0)]))
(1, RelationRecord(items=frozenset({'10-17', 'Black'}), support=0.051829268292682924, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'10-17', 'Black'}), confidence=0.051829268292682924, lift=1.0), OrderedStatistic(items_base=frozenset({'10-17'}), items_add=frozenset({'Black'}), confidence=0.25757575757575757, lift=1.4566353187042842), OrderedStatistic(items_base=frozenset({'Black'}), items_add=frozenset({'10-17'}), confidence=0.29310344827586204, lift=1.456635318704284)]))
(2, RelationRecord(items=frozenset({'18-24', 'Black'}), support=0.057926829268292686, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24', 'Black'}), confidence=0.057926829268292686, lift=1.0), OrderedStatistic(items_base=frozenset({'1

In [68]:
for v in police_black[8][2]:
  print(v)

OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Male', 'Article for use in theft', 'Black'}), confidence=0.14329268292682926, lift=1.0)
OrderedStatistic(items_base=frozenset({'Article for use in theft'}), items_add=frozenset({'Male', 'Black'}), confidence=0.16725978647686832, lift=0.9796644636502286)
OrderedStatistic(items_base=frozenset({'Black'}), items_add=frozenset({'Male', 'Article for use in theft'}), confidence=0.8103448275862069, lift=1.0848698099929626)
OrderedStatistic(items_base=frozenset({'Male'}), items_add=frozenset({'Article for use in theft', 'Black'}), confidence=0.16607773851590105, lift=1.1117040455758274)
OrderedStatistic(items_base=frozenset({'Article for use in theft', 'Black'}), items_add=frozenset({'Male'}), confidence=0.9591836734693877, lift=1.1117040455758274)
OrderedStatistic(items_base=frozenset({'Male', 'Article for use in theft'}), items_add=frozenset({'Black'}), confidence=0.19183673469387755, lift=1.0848698099929628)
OrderedStatistic(item

##

In [81]:
data_SS['Object of search'].unique()

array(['Offensive weapons', 'Controlled drugs', 'Stolen goods',
       'Fireworks', 'Articles for use in criminal damage',
       'Article for use in theft'], dtype=object)

In [92]:
oj =list()
for result in list_apri:
  if 'Controlled drugs' in result[0]:
    oj.append(result)

In [93]:
oj

[RelationRecord(items=frozenset({'Controlled drugs'}), support=0.08536585365853659, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Controlled drugs'}), confidence=0.08536585365853659, lift=1.0)]),
 RelationRecord(items=frozenset({'Male', 'Controlled drugs'}), support=0.06707317073170732, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Male', 'Controlled drugs'}), confidence=0.06707317073170732, lift=1.0), OrderedStatistic(items_base=frozenset({'Controlled drugs'}), items_add=frozenset({'Male'}), confidence=0.7857142857142857, lift=0.9106511862695609), OrderedStatistic(items_base=frozenset({'Male'}), items_add=frozenset({'Controlled drugs'}), confidence=0.07773851590106008, lift=0.9106511862695609)]),
 RelationRecord(items=frozenset({'White', 'Controlled drugs'}), support=0.054878048780487805, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'White', 'Controlled drugs'}), confidence=0

In [94]:
for rule in enumerate(oj):
  print(rule)

(0, RelationRecord(items=frozenset({'Controlled drugs'}), support=0.08536585365853659, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Controlled drugs'}), confidence=0.08536585365853659, lift=1.0)]))
(1, RelationRecord(items=frozenset({'Male', 'Controlled drugs'}), support=0.06707317073170732, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Male', 'Controlled drugs'}), confidence=0.06707317073170732, lift=1.0), OrderedStatistic(items_base=frozenset({'Controlled drugs'}), items_add=frozenset({'Male'}), confidence=0.7857142857142857, lift=0.9106511862695609), OrderedStatistic(items_base=frozenset({'Male'}), items_add=frozenset({'Controlled drugs'}), confidence=0.07773851590106008, lift=0.9106511862695609)]))
(2, RelationRecord(items=frozenset({'White', 'Controlled drugs'}), support=0.054878048780487805, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'White', 'Controlled drugs'}), con

In [96]:
for v in oj[1][2]:
  print(v)

OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Male', 'Controlled drugs'}), confidence=0.06707317073170732, lift=1.0)
OrderedStatistic(items_base=frozenset({'Controlled drugs'}), items_add=frozenset({'Male'}), confidence=0.7857142857142857, lift=0.9106511862695609)
OrderedStatistic(items_base=frozenset({'Male'}), items_add=frozenset({'Controlled drugs'}), confidence=0.07773851590106008, lift=0.9106511862695609)
